In [2]:
#!pip install cv2
#!pip install mediapipe
import cv2
import mediapipe as mp
import numpy as np

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
#calculate angles

def calculateAngles(a,b,c):
	a=np.array(a)	#first  11
	b=np.array(b)	#mid  13
	c=np.array(c)	#End  15	
	radians = np.arctan2(c[1]-b[1],c[0]-b[0]) - np.arctan2(a[1]-b[1],a[0]-b[0])
	angle = np.abs(radians*180.0/np.pi)
	if angle > 180.0:
		angle = 360-angle
	return angle


#VIDEO FEED
cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0 
stage = None

#setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,frame = cap.read()
        
        #detect stuff and render
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        #make detection
        results = pose.process(image)

        #recolor back to BRG
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)


        try:
            landmark = results.pose_landmarks.landmark
            shoulder = [landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmark[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmark[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmark[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmark[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmark[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

            #calculate Angle
            angle = calculateAngles(shoulder,elbow,wrist)

            #visualize text 
            cv2.putText(image,str(angle),tuple(np.multiply(elbow,[640,480]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
        
             # Curl counter logic
            if angle > 160:
                stage = "down"
            if angle < 30 and stage =='down':
                stage="up"
                counter +=1
                print(counter)
        
        
        except:
        	pass


         # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        #render detections
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,mp_drawing.DrawingSpec(color=(145,177,66),thickness=2,circle_radius=2),mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2))


        cv2.imshow('Mediapipe Feed',image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


KeyboardInterrupt: 

In [7]:
mp_drawing.DrawingSpec??